In [6]:
import requests
import pandas as pd

OVERPASS_URL = "http://overpass-api.de/api/interpreter"

def extract_category(tags):
    if "tourism" in tags:
        return f"tourism:{tags['tourism']}"
    if "historic" in tags:
        return f"historic:{tags['historic']}"
    if "leisure" in tags:
        return f"leisure:{tags['leisure']}"
    if "amenity" in tags:
        return f"amenity:{tags['amenity']}"
    return "other"

query = """
[out:json][timeout:25];
area["name"="İstanbul"]->.searchArea;
(
  node["tourism"~"attraction|museum"](area.searchArea);
  way["tourism"~"attraction|museum"](area.searchArea);
  relation["tourism"~"attraction|museum"](area.searchArea);

  node["historic"](area.searchArea);
  way["historic"](area.searchArea);
  relation["historic"](area.searchArea);

  node["leisure"="park"](area.searchArea);
  way["leisure"="park"](area.searchArea);
);
out center;
"""

response = requests.get(OVERPASS_URL, params={"data": query}, timeout=60)

if response.status_code != 200:
    raise Exception("Overpass API request failed")

data = response.json().get("elements", [])
print("Total raw elements:", len(data))

pois = []

for el in data:
    tags = el.get("tags", {})

    lat = el.get("lat", el.get("center", {}).get("lat"))
    lon = el.get("lon", el.get("center", {}).get("lon"))

    if lat is None or lon is None:
        continue

    pois.append({
        "poi_id": el["id"],
        "name": tags.get("name", "Unknown"),
        "category": extract_category(tags),
        "lat": lat,
        "lon": lon
    })

df_pois = pd.DataFrame(pois)

print("Final POIs:", len(df_pois))

df_pois.to_csv("../data/raw/pois_istanbul.csv", index=False)

df_pois.head()


Total raw elements: 4915
Final POIs: 4915


,poi_id,name,category,lat,lon
0,269497288,Barbaros Hayrettin Türbesi,historic:memorial,41.041923,29.006836
1,269709058,Kanburun Bahçesi,tourism:attraction,41.042587,29.003895
2,288789401,Erbilgin Yalısı,tourism:attraction,41.118917,29.068419
3,288789412,Besteci Faik Canselen'in Evi,tourism:attraction,41.118811,29.064165
4,415157636,Yılanlı Sütun,tourism:attraction,41.005665,28.975110
